In [1]:
from peft import LoraConfig, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neo-125m")



In [2]:
from peft import IA3Config

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

ia3_config = IA3Config(
    task_type="CAUSAL_LM", 
    feedforward_modules=["down_proj"]
)

In [3]:
peft_model = get_peft_model(model, peft_config)
peft_model.print_trainable_parameters()

trainable params: 589,824 || all params: 125,788,416 || trainable%: 0.4689016832837771


In [4]:
peft_model = get_peft_model(model, ia3_config)
peft_model.print_trainable_parameters()

trainable params: 27,648 || all params: 125,226,240 || trainable%: 0.02207843979025482


In [5]:

from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
dataset = load_dataset("imdb", split="train")


/home/sallinen/miniconda3/envs/LLM/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [6]:
dataset = dataset.select(range(20))

In [7]:
from transformers import TrainingArguments

trainer = SFTTrainer(
    model,
    train_dataset=dataset,
    dataset_text_field="text",
    peft_config=ia3_config,
    args = TrainingArguments(
        output_dir="./test",
        num_train_epochs=10,
        per_device_train_batch_size=5,)
)

/home/sallinen/miniconda3/envs/LLM/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


In [8]:
    trainer.train()

wandb: Currently logged in as: alex-sallinen (alexs-team). Use `wandb login --relogin` to force relogin


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


TrainOutput(global_step=40, training_loss=3.7536869049072266, metrics={'train_runtime': 487.6507, 'train_samples_per_second': 0.41, 'train_steps_per_second': 0.082, 'total_flos': 60538608000000.0, 'train_loss': 3.7536869049072266, 'epoch': 10.0})